In [ ]:
import os
import shutil
from pathlib import Path

from IPython.display import Image, display

import matplotlib.pyplot as plt
import numpy as np
import obspy

In [ ]:
# Define root_dir only if it is not already defined. This way root_dir stays the same, even if you change the work directory.
try:
    root_dir
except NameError:
    root_dir = Path(os.curdir).absolute()

# SPECFEM bin directory
SPECFEM2D_BIN = root_dir / "specfem2d" / "bin"
# Simulation directory
run_folder = root_dir / "01_Default_Model"
# Go into run directory
os.chdir(run_folder)
# More readable paths
run_folder = run_folder.relative_to(Path(".").absolute())

# Running A Simulation

Create a clean `OUTPUT_FILES` directory.

In [ ]:
output_folder = run_folder / "OUTPUT_FILES"

if output_folder.exists():
    shutil.rmtree(output_folder)

output_folder.mkdir()

Running the mesher

In [ ]:
! $SPECFEM2D_BIN/xmeshfem2D

In [ ]:
! $SPECFEM2D_BIN/xspecfem2D

# Outputs

### Seismograms

Seismograms are written in `OUTPUT_FILES` directory. Since in this simulation, we are working with displacements and ascii outputs, it created files with `.semd` extension.

In [ ]:
seismograms = sorted(output_folder.glob("*.semd"))
print("Example files:")
seismograms[:5]

Seismogram files are `ascii` files with columns representing time and values.

In [ ]:
seismogram_file = seismograms[0]
print(f"{seismogram_file}:")
! head $seismogram_file
print("...")

#### SAC

For quick look, you can open them using `sac`

```console
$ sac
 SEISMIC ANALYSIS CODE [09/11/2019 (Version 101.6)]
 Copyright 1995 Regents of the University of California

SAC> readtable content p ./OUTPUT_FILES/AA.S0001.BXZ.semd
SAC> p1
```

![Plot of a seismogram using sac](./include/sacplot.png)

#### Using Python

In [ ]:
data = np.loadtxt(seismogram_file)
data

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))
ax.plot(data[:, 0], data[:, 1], "k")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Displacement")
ax.set_title(seismogram_file.stem)
fig.tight_layout()

In [ ]:
def read_ascii_trace(filename: Path):
    """Reads SPECFEM2D ascii file and returns a obspy.Trace"""
    data = np.loadtxt(filename)
    # Find deltat
    dt = data[:, 0][1]-data[:, 0][0]
    net, sta, comp = filename.stem.split(".")
    
    stats = {"delta": dt, "network": net, "station": sta, "channel": comp, "b": data[0, 0]}
    return obspy.Trace(data[:, 1], stats)

In [ ]:
tr = read_ascii_trace(seismogram_file)
tr.plot();

# Wavefield Plots

In [ ]:
wavefield_plots = sorted(output_folder.glob("*.jpg"))

In [ ]:
for wavefield_plot in wavefield_plots:
    display(Image(filename=wavefield_plot))